### Подключаем пакет

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

## Combine_first

In [2]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
display(a)
display(b)
np.where(pd.isnull(a), b, a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [3]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [4]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
display(df1)
display(df2)
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### Stack and Unstack

In [5]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [7]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [8]:
result.unstack(0)
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [9]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
display(data2)
data2.unstack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [10]:
display( data2.unstack() )
display( data2.unstack().stack() )
display( data2.unstack().stack(dropna=False) )

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [11]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
display( df )
df.unstack('state')

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [12]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Melt / Pivot

In [13]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [14]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [15]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [16]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [17]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [18]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [19]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


## Агрегирование

In [20]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.921029,-0.075296
1,a,two,-1.344499,-0.091299
2,b,one,0.537170,0.153251
3,b,two,1.651970,-0.753991
4,a,one,-0.483419,-0.496584


In [21]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [22]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.218805
      two    -1.344499
b     one     0.537170
      two     1.651970
Name: data1, dtype: float64

In [23]:
means.unstack()

key2,one,two
key1,,
a,0.218805,-1.344499
b,0.537170,1.651970


In [24]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -1.344499
            2006    0.537170
Ohio        2005    1.286500
            2006   -0.483419
Name: data1, dtype: float64

In [25]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.302296,-0.221059
b,1.094570,-0.300370


In [26]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.218805 -0.285940
     two  -1.344499 -0.091299
b    one   0.537170  0.153251
     two   1.651970 -0.753991

In [27]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [28]:
for name, group in df.groupby('key1'):
    display(name)
    display(group)

'a'

,key1,key2,data1,data2
0,a,one,0.921029,-0.075296
1,a,two,-1.344499,-0.091299
4,a,one,-0.483419,-0.496584


'b'

,key1,key2,data1,data2
2,b,one,0.53717,0.153251
3,b,two,1.65197,-0.753991


In [29]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    display((k1, k2))
    display(group)

('a', 'one')

,key1,key2,data1,data2
0,a,one,0.921029,-0.075296
4,a,one,-0.483419,-0.496584


('a', 'two')

,key1,key2,data1,data2
1,a,two,-1.344499,-0.091299


('b', 'one')

,key1,key2,data1,data2
2,b,one,0.53717,0.153251


('b', 'two')

,key1,key2,data1,data2
3,b,two,1.65197,-0.753991


In [30]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.53717,0.153251
3,b,two,1.65197,-0.753991


In [31]:
df.dtypes
grouped = df.groupby(df.dtypes, axis=1)

In [32]:
for dtype, group in grouped:
    display(dtype)
    display(group)

dtype('float64')

,data1,data2
0,0.921029,-0.075296
1,-1.344499,-0.091299
2,0.537170,0.153251
3,1.651970,-0.753991
4,-0.483419,-0.496584


dtype('O')

,key1,key2
0,a,one
1,a,two
2,b,one
3,b,two
4,a,one


In [33]:
display(df)
df.groupby(['key1', 'key2'])[['data2']].mean()

,key1,key2,data1,data2
0,a,one,0.921029,-0.075296
1,a,two,-1.344499,-0.091299
2,b,one,0.537170,0.153251
3,b,two,1.651970,-0.753991
4,a,one,-0.483419,-0.496584


data2
key1 key2          
a    one  -0.285940
     two  -0.091299
b    one   0.153251
     two  -0.753991

In [34]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.514387,0.800211,0.433545,0.032348,-1.234215
Steve,1.542407,0.354673,0.526498,0.486457,-0.860680
Wes,2.050572,NaN,NaN,-0.418136,-0.389922
Jim,1.505168,1.761131,-0.964667,-0.829826,-1.999892
Travis,-0.366471,-0.159235,0.223673,0.096845,0.481831


In [35]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [36]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.465893,-0.948392
Steve,1.012956,1.036400
Wes,-0.418136,1.660650
Jim,-1.794494,1.266407
Travis,0.320518,-0.043875


In [37]:
map_series = pd.Series(mapping)
display(map_series)
people.groupby(map_series, axis=1).count()

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [38]:
display(people)
people.groupby(len).sum()

,a,b,c,d,e
Joe,-0.514387,0.800211,0.433545,0.032348,-1.234215
Steve,1.542407,0.354673,0.526498,0.486457,-0.860680
Wes,2.050572,NaN,NaN,-0.418136,-0.389922
Jim,1.505168,1.761131,-0.964667,-0.829826,-1.999892
Travis,-0.366471,-0.159235,0.223673,0.096845,0.481831


,a,b,c,d,e
3,3.041353,2.561342,-0.531122,-1.215615,-3.624030
5,1.542407,0.354673,0.526498,0.486457,-0.860680
6,-0.366471,-0.159235,0.223673,0.096845,0.481831


In [39]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.514387  0.800211  0.433545 -0.418136 -1.234215
  two  1.505168  1.761131 -0.964667 -0.829826 -1.999892
5 one  1.542407  0.354673  0.526498  0.486457 -0.860680
6 two -0.366471 -0.159235  0.223673  0.096845  0.481831

In [40]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.334188 -0.804911 -0.410744  1.139550 -1.214372
1      0.268238  0.844134 -0.680633 -1.643685  0.424311
2      2.734304  0.315122  1.620023 -0.249636 -0.669349
3     -2.245263 -0.359493 -1.168070  1.982288 -0.593254

In [41]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [42]:
display(df)
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

,key1,key2,data1,data2
0,a,one,0.921029,-0.075296
1,a,two,-1.344499,-0.091299
2,b,one,0.537170,0.153251
3,b,two,1.651970,-0.753991
4,a,one,-0.483419,-0.496584


key1
a    0.64014
b    1.54049
Name: data1, dtype: float64

In [43]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.265529,0.421288
b,1.114800,0.907242


In [44]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.302296  1.143573 -1.344499 -0.913959 -0.483419  0.218805   
b      2.0  1.094570  0.788283  0.537170  0.815870  1.094570  1.373270   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.921029   3.0 -0.221059  0.238745 -0.496584 -0.293941 -0.091299   
b     1.651970   2.0 -0.300370  0.641517 -0.753991 -0.527181 -0.300370   

                          
           75%       max  
key1                      
a    -0.083297 -0.075296  
b    -0.073560  0.153251